In [1]:
from sympy import Matrix, Symbol, solve, Eq, zeros

In [2]:
n = 3
A = Matrix(n, n, [2, 6, 5, -3, 8, 2, 6, 8, 1])
A

Matrix([
[ 2, 6, 5],
[-3, 8, 2],
[ 6, 8, 1]])

In [3]:
b = Matrix([2,5,7])

In [4]:
L = Matrix(n, n, [Symbol(f"L{i},{j}") for i in range(n) for j in range(n)])
L

Matrix([
[L0,0, L0,1, L0,2],
[L1,0, L1,1, L1,2],
[L2,0, L2,1, L2,2]])

In [5]:
U = Matrix(n, n, [Symbol(f"U{i},{j}") for i in range(n) for j in range(n)])
U

Matrix([
[U0,0, U0,1, U0,2],
[U1,0, U1,1, U1,2],
[U2,0, U2,1, U2,2]])

In [6]:
for i in range(n):
    for j in range(n):
        if j > i:
            L[i, j] = 0
        if i > j:
            U[i, j] = 0

U

Matrix([
[U0,0, U0,1, U0,2],
[   0, U1,1, U1,2],
[   0,    0, U2,2]])

In [7]:
L

Matrix([
[L0,0,    0,    0],
[L1,0, L1,1,    0],
[L2,0, L2,1, L2,2]])

In [8]:
L[0, 0] = 4
L[1, 1] = 8
L[2, 2] = 12

C = L*U
C

Matrix([
[   4*U0,0,                4*U0,1,                          4*U0,2],
[L1,0*U0,0,    L1,0*U0,1 + 8*U1,1,              L1,0*U0,2 + 8*U1,2],
[L2,0*U0,0, L2,0*U0,1 + L2,1*U1,1, L2,0*U0,2 + L2,1*U1,2 + 12*U2,2]])

In [9]:
for i in range(n):
    U[0, i] = solve(Eq(C[0, i], A[0, i]), U[0, i])

U

Matrix([
[1/2,  3/2,  5/4],
[  0, U1,1, U1,2],
[  0,    0, U2,2]])

In [10]:
for i in range(1, n):
    for j in range(n):
        if i > j:
            L[i, j] = solve(Eq(C[i, j], A[i, j]), L[i, j])
        else:
            U[i, j] = solve(Eq(C[i, j], A[i, j]), U[i, j])

sols = {Symbol(f"U{i},{j}"):U[i,j] for i in range(n) for j in range(n)} | {Symbol(f"L{i},{j}"):L[i,j] for i in range(n) for j in range(n)}

U = U.subs(sols)
L = L.subs(sols)

sols = {Symbol(f"U{i},{j}"):U[i,j] for i in range(n) for j in range(n)} | {Symbol(f"L{i},{j}"):L[i,j] for i in range(n) for j in range(n)}


In [11]:
L = L.subs(sols)
L

Matrix([
[ 4,      0,  0],
[-6,      8,  0],
[12, -80/17, 12]])

In [12]:
U = U.subs(sols)
U

Matrix([
[1/2,  3/2,      5/4],
[  0, 17/8,    19/16],
[  0,    0, -143/204]])

In [13]:
C = L.inv() * b
C

Matrix([
[   1/2],
[     1],
[97/204]])

In [14]:
E = U.col_insert(3, C)
E

Matrix([
[1/2,  3/2,      5/4,    1/2],
[  0, 17/8,    19/16,      1],
[  0,    0, -143/204, 97/204]])

In [15]:
def backsubs(A, b):
    A = A.copy()
    b = b.copy()
    
    n = b.shape[0]

    xcomp = Matrix([0 for _ in range(n)])

    for i in range(n-1, -1, -1):
        tmp = b[i]
        for j in range(n-1, i, -1):
            tmp -= xcomp[j]*A[i,j]
            
        xcomp[i] = tmp/A[i,i]

    return xcomp

In [16]:
sol = backsubs(U, C)
sol

Matrix([
[ 21/143],
[243/286],
[-97/143]])

# Comprobar

In [17]:
A.gauss_jordan_solve(b)[0]

Matrix([
[ 21/143],
[243/286],
[-97/143]])